In [1]:
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

from classevy.klas import StudentGroup, PlanPopulation, Plan

In [2]:
students = StudentGroup('../data/students.csv')

In [4]:
students[:6]

,name,score_math,score_spelling,gender,learning_advantage,not_together,together,preferences
number,,,,,,,,
1,luuk,10,8,1,1,"(4,)","(2,)","(3, 6, 17)"
2,pat,9,7,1,0,(),"(1,)","(5, 9, 27)"
3,jos,8,6,1,0,"(6,)",(),"(2, 1, 21)"
4,pol,7,9,1,0,"(1,)",(),"(5, 6, 7)"
5,katrien,6,7,-1,0,(),(),"(6, 7, 23)"
6,mo,5,5,1,0,"(3, 8)",(),"(12, 5, 25)"


In [5]:
plan = Plan(students,2)

In [10]:
plan.print_summary()

Mean classes_score_math per class: [6.666666666666667, 7.266666666666667]
Spread of mean spread_score_math over classes: 0.2999999999999998
Mean classes_score_spelling per class: [6.583333333333333, 6.733333333333333]
Spread of mean spread_score_spelling over classes: 0.07500000000000018
Mean classes_gender per class: [0.16666666666666666, 0.4666666666666667]
Spread of mean spread_gender over classes: 0.15000000000000002
Mean classes_learning_advantage per class: [0.08333333333333333, 0.0]
Spread of mean spread_learning_advantage over classes: 0.041666666666666664


In [11]:
def hypo_spread(num, classes):
    '''Hypothetically assigning this num to one class or the other would result in which new spreads?
    We want to assign it to the option which causes the smallest new spread.
    '''
    classes = [k if len(k) else [0] for k in classes]
    mean_stds = [] # will contain all possible std when the num is added to a class
    for c in classes:
        classes_no_c = classes.copy()
        classes_no_c.remove(c)
        mean_stds.append(np.array([np.array(c0).mean() for c0 in classes_no_c]+[np.array(c+[num]).mean()]).std())
    return mean_stds

hypo_spread(1, [[-2, -2],[1, 1]])

[1.0, 1.5]

In [44]:
def biggest_impact(num_list, classes):
    '''Which num in num_list has the biggest impact on the spread?
    Pop it from the num_list
    '''
    classes = [k if len(k) else [0] for k in classes]
    max_stds = []
    for i, num in enumerate(num_list):
        #print('num',num)
        mean_stds = hypo_spread(num, classes)
        #print('new hypo std', mean_stds)
        max_stds.append(max(mean_stds))
    #print('hypo stds:', mean_stds)
    index = np.argmax(np.array(max_stds))
    #print('pick', index)
    return num_list.pop(index)

biggest_impact([1, -1], [[1],[0]])

-1

In [45]:
def pop_absmax(l):
    absmax = max([abs(i) for i in l])
    if absmax in l:
        m = absmax
    elif -absmax in l:
        m = -absmax
    
    index = l.index(m)
    return l.pop(index)

a = [1,-2]
pop_absmax(a)

-2

In [57]:
def divide_one_prop(plan, prop):
    num_list = list(plan.students[prop].values)
    classes = [[] for _ in plan.classes]
    for i in range(len(num_list)):

        print('means', [np.array(c).mean() for c in classes])

        # pick the number with the highest potential impact:
        num = biggest_impact(num_list, classes)
        print('num:', num)
        # if any of the classes is still empty, add it there:
        if not all([len(c) for c in classes]):
            for c in classes:
                if len(c) == 0:
                    c.append(num)
                    break
        else:
            #print('hypo new means', [np.array(c+[num]).mean() for c in classes])
            mean_stds = hypo_spread(num, classes)
            # now add the num to the class that results in the smallest std:
            #print('mean_stds:', mean_stds)
            index = np.argmin(np.array(mean_stds))
            #print('picking', index)
            classes[index].append(num)
    means = [np.array(c).mean() if len(c) else 0 for c in classes]
    spread = np.array(means).std()
    return classes, means, spread
classes, means, spread = divide_one_prop(plan, 'gender')

print('means:', means)
print('spread:', spread)

means [nan, nan]
num: 1
means [1.0, nan]
num: -1
means [1.0, -1.0]
num: 1
means [1.0, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.3333333333333333, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.2, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.14285714285714285, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.1111111111111111, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.09090909090909091, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.07692307692307693, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.06666666666666667, 0.0]
num: -1
means [0.0, 0.0]
num: 1
means [0.058823529411764705, 0.0]
num: 1
means [0.1111111111111111, 0.0]
num: 1
means [0.15789473684210525, 0.0]
num: 1
means [0.15789473684210525, 0.3333333333333333]
num: 1
means [0.2, 0.3333333333333333]
num: 1
means [0.23809523809523808, 0.3333333333333333]
num: 1
means [0.2727272727272727, 0.3333333333333333]
num: 1
means [0.30434782608695654, 0.3333333333333333]
num: 1
means: [0.3333333333333333, 0.3333333333333333]
spread: 0.0


/var/folders/ph/hfm4py7x2rq6pcdmtsszvfmh0000gn/T/ipykernel_30447/1767088126.py:6: RuntimeWarning: Mean of empty slice.
  print('means', [np.array(c).mean() for c in classes])


In [ ]:
np.array(classes[0]).mean()

In [ ]:
np.array(classes[1]).mean()

In [ ]:
np.array(classes[0]+classes[1]).mean()

In [ ]:
len(ordered)

In [ ]:
len(classes[0])

In [ ]:
len(classes[1])

In [26]:
classes[0]

[1,
 -1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [27]:
classes[1]

[]

In [ ]:
20/3

In [51]:
np.array([4/12, 3/13]).std()

0.051282051282051266

In [52]:
np.array([5/13, 3/13]).std()

0.07692307692307693

In [53]:
np.array([4/12, 4/14]).std()

0.023809523809523808

In [54]:
np.array([4/12, 5/15]).std()

0.0